In [1]:
# Will
# 4.30
# GPUCB with beerslaw

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

from scipy.stats import pearsonr
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity

from importlib import reload
import itertools

from sklearn.gaussian_process import GaussianProcessRegressor
import sklearn.metrics

import beers_law.physics_models
import UCB.ucb

import tensorflow as tf
from tensorflow.keras import layers

In [3]:
def noise(size):
    return np.random.normal(loc=0, scale=1.5, size=size)

def make_solution(amts, dye):
    """
    Calculates concentration from volumes and dye concentrations
    """
    n = noise(4)
    
    red_ul = amts[0] + n[0]
    green_ul = amts[1] + n[1]
    blue_ul = amts[2] + n[2]
    water_ul = amts[3] + n[3]
    
    #makign sure we aren't adding negative volumes
    if red_ul < 0:
        red_ul = 0
    if green_ul < 0:
        green_ul = 0
    if blue_ul < 0:
        blue_ul = 0
    if water_ul < 0:
        water_ul = 0
    
    
    total = red_ul + green_ul + blue_ul + water_ul
    
    if total == 0:
        return {'red': 0, 'green': 0, 'blue': 0}
    
    
    red = red_ul * dye['red'] / total
    green = green_ul * dye['green'] / total
    blue = blue_ul * dye['blue'] / total
    return {'red': red, 'green': green, 'blue': blue}

In [4]:
def plot_beers_batches(data, target):
    """
    Plots each batch and target from agent
    """
    fig, axs = plt.subplots(nrows=len(data), ncols=1, figsize=(8, len(data) * 4))
    axs = axs.flatten()
    best_mse_guess = None
    best_mse = 100
    best_cos_guess = None
    best_cos = 0
    best_mix_guess = None
    best_mix = 100
    count = 0
    for batch in data:
        axs[count].plot(target[0], target[1], color='black', alpha=.8)
        for amts in batch:
            sol = make_solution(amts, dye)
            guess = beers_law.physics_models.beers_law(sol, 1, coeff, 300, 700)[1]
            mse = sklearn.metrics.mean_squared_error(guess, target[1])
            cos = sklearn.metrics.pairwise.cosine_similarity([target[1]], [guess])
            mix = mse * (1-cos)
            
            if mse < best_mse:
                best_mse_guess = guess
                best_mse = mse
            if cos > best_cos:
                best_cos_guess = guess
                best_cos = cos
            if mix < best_mix:
                best_mix_guess = guess
                best_mix = mix
            
            
            axs[count].plot(target[0], guess, alpha=.25)

        count += 1
    
    print(best_mse)
    print(best_cos)
    print(best_mix)
    plt.figure()
    plt.plot(target[0], target[1], color='black', label='target')
    plt.plot(target[0], best_cos_guess, label='best cos')
    plt.plot(target[0], best_mse_guess, label='best MSE')
    plt.plot(target[0], best_mix_guess, label='best mixed')
    plt.legend()

In [5]:
class BeersEnvironment(object):
    def __init__(self, target, coeffs, dye):
        # in future this will be basis spectra
        self.target = target
        self.coeffs = coeffs
        self.dye = dye
    
    def sample(self, action):
        sol = make_solution(action, self.dye)
        result = beers_law.physics_models.beers_law(sol, 1, coeff, 300, 700)[1]
        mse = sklearn.metrics.mean_squared_error(result, self.target)
        cos = sklearn.metrics.pairwise.cosine_similarity([result], [self.target])
        #print(cos)
        #print(-mse)
        #return cos
        return -mse
        return pearsonr(result, self.target)[0]

In [6]:
red = np.arange(0, 301, 6)
green = np.arange(0, 301, 6)
blue = np.arange(0, 301, 6)
water = np.arange(0, 301, 6)
grid = np.meshgrid(red, green, blue, water)

coeff = pd.read_csv('beers_law/rgb_coeff')
coeff.drop(labels='Unnamed: 0', axis=1, inplace=True)
spectra = beers_law.physics_models.random_spectra(1, coeff, 300, 700, .3)

dye = {'red': 5, 'blue': 5, 'green': 5}

env = BeersEnvironment(spectra[1], coeff, dye)
batch_size = 15

agent = UCB.ucb.BatchGPUCB(batch_size, grid, env, 5)
agent_rand = UCB.ucb.BatchGPUCB(batch_size, grid, env, 5)

In [7]:
#for i in range(5):
    #agent.learn()
    #agent_rand.learn_with_random()
    #print('----')

In [8]:
shape = np.array(grid).shape[1:]
np.log(np.prod(shape))
np.array(grid).shape

(4, 51, 51, 51, 51)

In [36]:
class SmartBatchGPUCB(UCB.ucb.BatchGPUCB):
    def __init__(self, input_shape, optimizer, n, *args, **kwargs):
        """
        """
        super().__init__(*args, **kwargs)
        
        last_init = tf.random_uniform_initializer()
        
        shape = self.meshgrid.shape[1:]
        output_shape = len(shape) ** n
        inputs = layers.Input(shape=input_shape)
        out = layers.Dense(input_shape*2, activation="relu")(inputs)
        out = layers.Dense(output_shape*2, activation="relu")(out)
        outputs = layers.Dense(output_shape, activation="relu")(out)
        outputs = -outputs
        
        self.guesser = tf.keras.Model(inputs, outputs)
        self.optimizer = optimizer
        
        self.guess_locs = self.get_guesser_locs(n)
        
        
    def guesser_learn(self, target):
        with tf.GradientTape() as tape:
            guess = self.guesser(tf.convert_to_tensor([target]), training=True)
            print(guess)
            loss = tf.Variable(self.guesser_loss(guess), trainable=True)
            grad = tape.gradient(loss, self.guesser.trainable_variables)
            self.optimizer.apply_gradients(
                zip(grad, self.guesser.trainable_variables)
            )
       
    
    def guesser_loss(self, guess):
        gp = GaussianProcessRegressor()
        Y = tf.make_ndarray(tf.make_tensor_proto(guess))[0]
        X = np.array(self.guess_locs).reshape(-1, self.input_dimension)
        gp.fit(self.guess_locs, Y)
        mus = gp.predict(self.X_grid, return_std=False)
        #np.mean(np.power((self.mu - mus), 2))
        
        return np.mean(np.power((self.mu - mus), 2))

    
    def get_guesser_locs(self, n):
        #hard coding because I can't figure this out lol
        X = np.array([
            [17, 17, 17, 17],
            [17, 17, 17, 34],
            [17, 17, 34, 17],
            [17, 17, 34, 34],
            [17, 34, 17, 17],
            [17, 34, 17, 34],
            [17, 34, 34, 17],
            [17, 34, 34, 34],
            [34, 17, 17, 17],
            [34, 17, 17, 34],
            [34, 17, 34, 17],
            [34, 17, 34, 34],
            [34, 34, 17, 17],
            [34, 34, 17, 34],
            [34, 34, 34, 17],
            [34, 34, 34, 34]]
        )
        return X

        
    def reset_with_guesser(self, target):
        X = self.guess_locs
        Y = self.model(target)
        
        gp = GaussianProcessRegressor()
        gp.fit(X, Y)
        
        self.mu = gp.predict(self.X_grid, return_std=False)
        self.sigma = np.array([0.5 for _ in range(self.X_grid.shape[0])])
        self.X = []
        self.Y = []
        self.T = 0
        pass
    
    
    def reset_with_prev(self, env):
        self.env = env
        self.X = []
        self.Y = []
        self.T = 0
        
    def resest_with_all(self):
        self.T = 0

In [37]:
optimizer = tf.keras.optimizers.Adam(.01)

guesser = SmartBatchGPUCB(len(spectra[1]), optimizer, 2, batch_size, grid, env, 5)
agent = UCB.ucb.BatchGPUCB(batch_size, grid, env, 5)

In [38]:
dumb_Xs = []
smart_Xs = []
target = beers_law.physics_models.random_spectra(1, coeff, 300, 700, .3)[1]


for a in range(5):
    for i in range(1):
        print('iteration:', a,i)
        #guesser.learn()
        agent.learn()

    guesser.guesser_learn(target)
    
    smart_Xs.append(guesser.X)
    dumb_Xs.append(agent.X)
    
    target = beers_law.physics_models.random_spectra(1, coeff, 300, 700, .3)[1]
    env = BeersEnvironment(spectra[1], coeff, dye)
    agent.reset_with_prev(env)
    #guesser.reset_with_guesser(target)
    
    

iteration: 0 0
tf.Tensor(
[[-0.01872934 -0.06284136 -0.19667791 -0.         -0.         -0.06812462
  -0.0072501  -0.26806736 -0.         -0.         -0.         -0.
  -0.         -0.07024486 -0.05899392 -0.        ]], shape=(1, 16), dtype=float32)


ValueError: No gradients provided for any variable: ['dense_27/kernel:0', 'dense_27/bias:0', 'dense_28/kernel:0', 'dense_28/bias:0', 'dense_29/kernel:0', 'dense_29/bias:0'].